## Import libraries

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 21.7 MB/s eta 0:00:00


In [14]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

## Autoencoder

In [3]:
class Encoder(nn.Module):

  def __init__(self):
    super().__init__()
    self.l1 = nn.Sequential(nn.Linear(28*28,64),nn.ReLU(),nn.Linear(64,3))

  def forward(self,x):
    return self.l1(x)

class Decoder(nn.Module):

  def __init__(self):
    super().__init__()
    self.l2 = nn.Sequential(nn.Linear(3,64),nn.ReLU(),nn.Linear(64,28*28))

  def forward(self,x):
    return self.l2(x)

In [4]:
class LiTAutoEncoder(L.LightningModule):

  def __init__(self,encoder,decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def training_step(self,batch,batch_idx):
    x,y = batch
    x = x.view(x.size(0),-1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    loss = F.mse_loss(x_hat,x)
    return loss

  def test_step(self,batch,batch_idx):
    x,y = batch
    x = x.view(x.size(0),-1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    loss = F.mse_loss(x_hat,x)
    self.log("test_loss",loss)

  def validation_step(self,batch,batch_idx):
    x,y = batch
    x = x.view(x.size(0),-1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    loss = F.mse_loss(x_hat,x)
    self.log("validation_loss",loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(),lr=1e-3)
    return optimizer

In [5]:
train_dataset = MNIST(os.getcwd(),download=True,transform=transforms.ToTensor(),train=True)
test_dataset = MNIST(os.getcwd(),download=True,transform=transforms.ToTensor(),train=False)

train_size = int(len(train_dataset)*0.8)
val_size = int(len(train_dataset)) - train_size
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[train_size,val_size])

train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 154482464.71it/s]

Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27562160.14it/s]


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 44862649.10it/s]

Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 21453298.16it/s]


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



In [16]:
autoencoder = LiTAutoEncoder(Encoder(),Decoder())
trainer = L.Trainer(max_epochs=10,default_root_dir = '/content/output',callbacks=[EarlyStopping(monitor='validation_loss',mode='min',patience=2)])
trainer.fit(autoencoder,train_loader,val_loader,ckpt_path = '/content/output/lightning_logs/version_0/checkpoints/epoch=0-step=1500.ckpt')
trainer.test(model = autoencoder,dataloaders = test_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Restoring states from the checkpoint path at /content/output/lightning_logs/version_0/checkpoints/epoch=0-step=1500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/output/lightning_logs/version_0/checkpoints/epoch=0-step=1500.ckpt
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:347: The dirpath has changed from '/content/output/lightning_logs/version_0/checkpoints' to '/content/output/lightning_lo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03854241594672203    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.03854241594672203}]

## Loading the mode

In [7]:
autoencoder = LiTAutoEncoder(Encoder(),Decoder())
optimizer = autoencoder.configure_optimizers()

for batch_idx,batch in enumerate(train_loader):
  loss = autoencoder.training_step(batch,batch_idx)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()